In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In this notebook, an attempt to get only the most important characters from south park, without any prior knowledge regarding the show, will be made.  

These attempt are based on network metrics, centrality measures and degree distributions.  

We will then dispose of the previous network and create a new one, using only the important nodes.

First, we load the data from the characters dataframe and the previously created netowrk, with all characters and nodes form fandom.

In [2]:
# Load dataframe
characters_df = pd.read_csv('characters_df_large.csv')

# Load network
with open('south_park_digraph.pickle', 'rb') as f:
    G = pickle.load(f)

The main idea for getting rid of the 'noise', or keeping only the most important characters, is to look at the characters' 'connectedness' using different criteria:  
- Degree (total, or in and out)
- Centrality measures:  
  - Closeness  
  - Betweenness  
  - Eigenvector based.
  
We will look at all measures, because each one represents different properties. In the ned, we can either normalize and take a weighted average of all the criteria for each character, and keep the top percentile, or use these metrics and extract the chaarcters with a different startegy.

In [3]:
def return_centrality_sorted(centrality_dict, top_n = None):
    a = sorted( centrality_dict.items(), key=lambda x: x[1], reverse=True )
    if top_n:
        return a[:top_n]
    return a

#### Betweenness centrality:  

A node has high betweenness centrality if many shortest paths between the other nodes in the network pass through it. High betweenness centrality for a node measn that a lot of information passess through this node, so it is bound to play a central role in the series.

In [4]:
betweenness = nx.betweenness_centrality(G.to_undirected(), normalized=True)
return_centrality_sorted(betweenness, top_n=10)

[('Eric Cartman', 0.2655766515997768),
 ('Stan Marsh', 0.13896494221466524),
 ('Kyle Broflovski', 0.10836022860439216),
 ('Randy Marsh', 0.09806368708331918),
 ('Butters Stotch', 0.07892490799843789),
 ('Herbert Garrison', 0.06804477974477509),
 ('The Boys', 0.06799352133780456),
 ('Kenny McCormick', 0.05297638908276538),
 ('The 4th Grade', 0.03806317281053689),
 ('Jimmy Valmer', 0.02845081381096589)]

#### Eigenvector centrality:  

Takes the neighboring nodes into consideration. A node has high eigenvector centrality if its neighbors have high centrality.  It is a measure of the importance of the node, since the higher the centrality, the more connected to important nodes the node is.

In [5]:
eigenvector = nx.eigenvector_centrality(G.to_undirected())
return_centrality_sorted(eigenvector, top_n=10)

[('McNuggets', 0.14313313731241356),
 ('Elon Musk', 0.1423357627341641),
 ('ISIS', 0.14122043095826411),
 ('PewDiePie', 0.1411205578470931),
 ('Jeff White', 0.1405343827111894),
 ('Dan Snyder', 0.1402312422510927),
 ('NFL Owners and Presidents', 0.1400859319673538),
 ('Unnamed Interior Designer', 0.1400409547113547),
 ('Taylor Swift', 0.13996296497869212),
 ('Possessed Stan Marsh', 0.13992283574187606)]

#### Degree centrality:  

A measure of the number of nodes connected to the node.

In [6]:
degree_centr = nx.degree_centrality(G.to_undirected())
return_centrality_sorted(degree_centr, top_n=10)

[('Eric Cartman', 0.2622871046228711),
 ('Stan Marsh', 0.17420924574209246),
 ('Kyle Broflovski', 0.15766423357664233),
 ('Butters Stotch', 0.11824817518248176),
 ('Randy Marsh', 0.11435523114355231),
 ('Kenny McCormick', 0.09732360097323602),
 ('The Boys', 0.09099756690997568),
 ('Herbert Garrison', 0.08564476885644769),
 ('The 4th Grade', 0.07201946472019465),
 ('Wendy Testaburger', 0.04720194647201947)]

#### Closeness centrality:  
Nodes with high closeness centrality are also close to other nodes. Meaning that they will most probably have many short paths. 

In [7]:
closeness_centr = nx.closeness_centrality(G.to_undirected())
return_centrality_sorted(closeness_centr, top_n=10)

[('Eric Cartman', 0.5144044304150723),
 ('Stan Marsh', 0.4829465684114917),
 ('Kyle Broflovski', 0.4783676702121729),
 ('Butters Stotch', 0.45082205107858536),
 ('Kenny McCormick', 0.44485967273955923),
 ('Randy Marsh', 0.4443155684431557),
 ('The Boys', 0.4400101850280088),
 ('Herbert Garrison', 0.43443244290734767),
 ('Clyde Donovan', 0.41996810261170414),
 ('Wendy Testaburger', 0.41678798618531904)]

How can we combine the above metrics to pick the most important characters, since for each metric, the top list is not the same?   
First of all, we notice that only the eigenvector centrality is very different compared to the other centrality measures. But we cannot ignore it.  

An idea is to create an equally weighted mean of each centrality measure for each character.  

Another idea comes from the fact that some characters might just appear only once and then be connected to the most important characters for one episode, they might have very high eigenvector centrality. To avoid this, we multiply together all the measures for each character.  

Finally, we can find percentiles for each measure and select the characters that are above a certain percentile in all centrality measures.

We can create a pandas dataframe where the first column is the character name and then each column holds its cecntrality measure. Finally, in the last column the equally weighted mean is calculated:

In [17]:
data = {'name': list(betweenness.keys()),
        'betweenness':list(betweenness.values()),
        'eigenvector':list(eigenvector.values()),
        'degree': list(degree_centr.values()),
        'closeness':list(closeness_centr.values()),
       }

centralities_df = pd.DataFrame.from_dict(data)

In [18]:
cols = centralities_df.columns

In [49]:
percentiles = [50,50, 50,50] # Weigh betweenness more

centralities_perc = [ np.percentile( centralities_df[cols[i+1]], percentiles[i]) for i in range(4) ]

In [55]:
reduced_centralities_df = centralities_df[  (centralities_df[cols[1]] > centralities_perc[0])
                & (centralities_df[cols[2]] > centralities_perc[1]) 
                & (centralities_df[cols[3]] > centralities_perc[2]) 
                & (centralities_df[cols[4]] > centralities_perc[3]) 
            ]

In [56]:
reduced_centralities_df.sort_values('betweenness', ascending=False)

,name,betweenness,eigenvector,degree,closeness
0,Eric Cartman,0.265577,0.078864,0.262287,0.514404
3,Stan Marsh,0.138965,0.051882,0.174209,0.482947
2,Kyle Broflovski,0.108360,0.047553,0.157664,0.478368
1545,Randy Marsh,0.098064,0.032506,0.114355,0.444316
236,Butters Stotch,0.078925,0.026209,0.118248,0.450822
...,...,...,...,...,...
1935,Unified Atheist League (UAL),0.000011,0.002155,0.002433,0.342129
1568,Rex,0.000011,0.004621,0.002920,0.362765
1105,Mark Zuckerberg,0.000011,0.003944,0.003893,0.358965
707,Hackelm Cartman,0.000011,0.003489,0.003406,0.349435


What do we have as a result? Many characters which are indeed central (from someone that has seen the series) but also many characters which are:  
- Groups (eg. Coon and friends, The Boys)  
- Characters from one specific episode, marked with parentheses  
- Characters which have no names and are some other character's relatives. Usually with a `'`  

Assuming no prior knowledge of the show, one cannot know in advance if any of these aforementioned character groups should be removed. As a result, we continue.